<a href="https://colab.research.google.com/github/ysdede/asrtk/blob/main/notebooks/filter_PleIAs_YouTube_Commons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
import csv
import json

from datasets import load_dataset

KEY = "original_language"  # alt: transcription_language
VALUE = "es"
MAX_ROWS = 250  # Set the maximum number of rows to process, set None to process all rows.
DEBUG = False  # Set True to print rows, it will cause notebook crashes in long runs.

dataset = load_dataset("PleIAs/YouTube-Commons", streaming=True)
# dataset = dataset_raw.shuffle(seed=1, buffer_size=100_000)  # shuffle dataset to tackle broken parquet files. (did not worked)

print("Dataset loaded.")

matching_rows = []
idx = 0
skipped_rows = 0
exception_count = 0

def copy_truncate_pad(original_dict, keys=['title', 'text'], max_length=48):
    new_dict = original_dict.copy()
    for key in keys:
        if key in new_dict:
            truncated = new_dict[key][:max_length] if len(new_dict[key]) > max_length else new_dict[key]
            new_dict[key] = f"{truncated:<{max_length}}"
    return new_dict

# Iterate over the 'train' split of the dataset
for example in dataset['train']:
    try:  # try-catch not working for handling dataset errors!
        # Check if the key exists in the example
        if KEY in example:
            if example[KEY] == VALUE:
                # Add the matching example to the list
                matching_rows.append(example)
                if DEBUG:
                  print(copy_truncate_pad(example))
                else:
                  print(">", end="")
                  if len(matching_rows) % 120 == 0:
                    print()
        else:
            print(f"Skipping example at index {idx} due to missing '{KEY}' key")
            skipped_rows += 1
    except Exception as e:
        print(f"Skipping example at index {idx} due to unexpected error: {str(e)}")
        exception_count += 1

    if MAX_ROWS and len(matching_rows) >= MAX_ROWS:
        break

    if idx % 100_000 == 0 and idx > 0:
        print(f"\n{len(matching_rows)}@{idx}")

    idx += 1

print(f"\nSkipped {skipped_rows} rows due to missing '{KEY}' key")
print(f"Exception count: {exception_count}")
print(f"Matching rows count: {len(matching_rows)}")

filename = f"{KEY}-{VALUE}"
# Save the matching rows to a CSV file
with open(f"{filename}.csv", "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = list(matching_rows[0].keys())
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter="\t")
    writer.writeheader()
    writer.writerows(matching_rows)

# Save the matching rows to a TXT file
with open(f"{filename}.txt", "w", encoding="utf-8") as txtfile:
    for row in matching_rows:
        txtfile.write(str(row) + "\n")
